# High Performance Pandas

## Libraries and Setup

All of the code in this section will get us setup for the rest of the lecture and ensure the output from the lecture looks reasonable. 

In [1]:
## import some standard data science modules
import sys
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import textwrap

In [2]:
## Enable inline plotting for graphics
%matplotlib inline
## So all output comes through from Ipython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
## Get Version information
print(textwrap.fill(sys.version),'\n')
print("Pandas version: {0}".format(pd.__version__))
print("Matplotlib version: {0}".format(matplotlib.__version__))
print("Numpy version: {0}".format(np.__version__))
print("Seaborn version: {0}".format(sns.__version__))

3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 14:01:38)  [GCC 4.2.1
Compatible Clang 4.0.1 (tags/RELEASE_401/final)] 

Pandas version: 0.23.4
Matplotlib version: 3.0.2
Numpy version: 1.15.4
Seaborn version: 0.9.0


In [4]:
## set pandas row print output
pd.options.display.max_rows = 10

I will use relative paths to import the data and store results. You should identify and update your working directory so paths correctly point at the downloaded data files. You can check your working directory like so:

In [5]:
# Working Directory
import os
print("My working directory:\n" + os.getcwd())
# Set Working Directory (if needed)
os.chdir(r"/Users/wenhuizeng/Library/Mobile Documents/com~apple~CloudDocs/high performance/week5/retail_sales")
#print("My new working directory:\n" + os.getcwd())

My working directory:
/Users/wenhuizeng/High Performance


## Efficient memory usage

As you know, Python is limited by the system's available memory. If we are using far less than the available memory, we don't really care about this and proceed as you have previously learned. However, if we start operating near the 'edge' of our memory (data objects > 1/3 the total available memory), we may run out of memory! Now we need to start trying to write memory efficient code. 

Let's bring this back to the concept of 'high performance Python' coding. If we are doing data management with Pandas (Python), there are multiple strategies we can employ to optimize resource usage during the data management steps.

One resource is **total memory**. Here are some strategies to manage memory footprints: 
  
1.  Use the smallest dtype possible (downcast data types)
2.  Filter columns/rows that aren't needed upon data ingestion
5.  Use inplace modifications for your dataframes to avoid making too many copies
3.  Work on chunks of the data one at a time to preprocess, then store the results on disk (in hdf5)
4.  Work on chunks of the data one at a time to preprocess, then store the results on disk (in sqlite database perhaps)

## Import and examine data

Let's start by importing the patient data, then using some of these techniques. 

In [6]:
## ingest the patient data
patient = pd.read_csv('patient_demo.csv')
## add some continuous columns for later using numpy random number simulations :) 
np.random.seed(10)
patient['lab_value_1'] = \
    np.round(np.random.normal(size=100000,
                             loc=100,scale=30),
             2)
patient['lab_value_2'] = \
    np.round(np.random.normal(size=100000,
                             loc=10,scale=2),
             8)
## check out data
patient

,patient_id,hosp_id,age,gender,mortality,SES,lab_value_1,lab_value_2
0,100000,77,70,Male,Yes,Med,139.95,11.357018
1,100001,45,64,Male,No,Low,121.46,8.956167
2,100002,5,76,Male,No,Med,53.64,9.267209
3,100003,13,51,Male,No,High,99.75,13.125279
4,100004,70,76,Male,No,Low,118.64,9.034593
...,...,...,...,...,...,...,...,...
99995,199995,60,66,Male,No,Med,90.88,10.028221
99996,199996,14,71,Male,No,Low,105.57,10.628457
99997,199997,39,73,Male,Yes,Med,106.52,10.366011
99998,199998,21,60,Female,No,High,78.02,12.258343


In [7]:
## ingest the hospital data
hosp = pd.read_csv('hosp_demo.csv') 
hosp

,hosp_id,academic
0,1,Yes
1,2,Yes
2,3,Yes
3,4,Yes
4,5,Yes
...,...,...
95,96,Yes
96,97,Yes
97,98,No
98,99,No


Let's check out some details on the total memory footprint of this data:

In [8]:
## check the size of the data frame
patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
patient_id     100000 non-null int64
hosp_id        100000 non-null int64
age            100000 non-null int64
gender         100000 non-null object
mortality      100000 non-null object
SES            100000 non-null object
lab_value_1    100000 non-null float64
lab_value_2    100000 non-null float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.1+ MB


In [9]:
## take the time to do an exact memory profile:
patient.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
patient_id     100000 non-null int64
hosp_id        100000 non-null int64
age            100000 non-null int64
gender         100000 non-null object
mortality      100000 non-null object
SES            100000 non-null object
lab_value_1    100000 non-null float64
lab_value_2    100000 non-null float64
dtypes: float64(2), int64(3), object(3)
memory usage: 21.0 MB


In [10]:
## Check this by column
#skip the line
col_summary = \
    patient.memory_usage(deep=True)
col_summary
## Check the percentage
col_summary/col_summary.sum()

Index               80
patient_id      800000
hosp_id         800000
age             800000
gender         6119864
mortality      5917989
SES            6019847
lab_value_1     800000
lab_value_2     800000
dtype: int64

Index          0.000004
patient_id     0.036268
hosp_id        0.036268
age            0.036268
gender         0.277447
mortality      0.268295
SES            0.272913
lab_value_1    0.036268
lab_value_2    0.036268
dtype: float64

We could also look at the memory usage by data type to better understand what columns are taking up all of the memory here:

In [11]:
## check dtypes
patient.dtypes

patient_id       int64
hosp_id          int64
age              int64
gender          object
mortality       object
SES             object
lab_value_1    float64
lab_value_2    float64
dtype: object

In [12]:
## check each dtype memory usage
## code inspired by https://www.dataquest.io/blog/pandas-big-data/
for dtype in ['float','int','object']:
    selected_dtype = patient.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 0.51 MB
Average memory usage for int columns: 0.57 MB
Average memory usage for object columns: 4.31 MB


It looks like the integers take up essentially no space, the float takes up a bit more, and the object columns take up 90% on average. This is a bit misleading, the 'index' is being included with the ints, and it is very small. But the other ints are the same size as the floats as we saw above. 

Can we somehow change these (default) dtypes to get a smaller dataframe? 

### Dtypes in Pandas

You have previously learned about high-level dtypes in Pandas, but now let's focus on a lower level perspective....

Pandas by default will use 64 bit versions of both floats and integers, as we saw above after importing the patient data. However, do we really need 64 bits of precision on those values? Perhaps not...

*  An unsigned 32 bit integer will cover ID values up to 4,294,967,296

In [13]:
## 8 bits (one byte)
2**8
## 16 bits (2 bytes)
2**16
## 32bit integer (4 bytes)
2**32
## 64 bit integer (8 bytes)
2**64

256

65536

4294967296

18446744073709551616

Let's convert some of the integers to something smaller than 64 bits using `.to_numeric()`, then check their size again

In [14]:
## Convert the int64s to unsigned (positive ints)
patient['patient_id_small'] =\
    pd.to_numeric(patient['patient_id'],
                  downcast = 'unsigned')
#down cast my patient id as small as possible
patient['hosp_id_small'] =\
    pd.to_numeric(patient['hosp_id'],
                  downcast = 'unsigned')
patient['age_small'] =\
    pd.to_numeric(patient['age'],
                  downcast = 'unsigned')

The floats can also be coerced to a lower precision, usually we would only go down to 32 from 64. 

In [15]:
## Convert the float for lab value 1
patient['lab_value_1_small'] =\
    pd.to_numeric(patient['lab_value_1'],
                  downcast = 'float')

Now we can check out the dtypes and memory efficiency:

In [16]:
## dtypes
patient.select_dtypes('number').dtypes
## memory 
col_summary = \
    patient\
    .select_dtypes('number')\
    .memory_usage(deep=True)
col_summary
## Check the percentage
col_summary/col_summary.sum()

patient_id             int64
hosp_id                int64
age                    int64
lab_value_1          float64
lab_value_2          float64
patient_id_small      uint32
hosp_id_small          uint8
age_small              uint8
lab_value_1_small    float32
dtype: object

Index                    80
patient_id           800000
hosp_id              800000
age                  800000
lab_value_1          800000
lab_value_2          800000
patient_id_small     400000
hosp_id_small        100000
age_small            100000
lab_value_1_small    400000
dtype: int64

Index                0.000016
patient_id           0.159997
hosp_id              0.159997
age                  0.159997
lab_value_1          0.159997
lab_value_2          0.159997
patient_id_small     0.079999
hosp_id_small        0.020000
age_small            0.020000
lab_value_1_small    0.079999
dtype: float64

As a final consideration, let's check out the text data or the 'object' data. There is another dtype in Pandas called 'categorical', which is like the R 'Factors'. This dtype will take the unique values of a column, then map them to an integer. This is far more efficient for repeated data. 

In [17]:
## convert strings to categoricals
patient['gender_small'] =\
    patient['gender'].astype('category')
patient['mortality_small'] =\
    patient['mortality'].astype('category')
patient['SES_small'] =\
    patient['SES'].astype('category')

Now let's check out the resulting values:

In [18]:
## dtypes
patient.select_dtypes(exclude='number').dtypes
## memory 
col_summary = \
    patient\
    .select_dtypes(exclude='number')\
    .memory_usage(deep=True)
col_summary
## Check the percentage
col_summary/col_summary.sum()

#compare gender to gender_small, the storage reduce a lot from 33% to 5%.

gender               object
mortality            object
SES                  object
gender_small       category
mortality_small    category
SES_small          category
dtype: object

Index                   80
gender             6119864
mortality          5917989
SES                6019847
gender_small        100204
mortality_small     100199
SES_small           100261
dtype: int64

Index              0.000004
gender             0.333354
mortality          0.322358
SES                0.327906
gender_small       0.005458
mortality_small    0.005458
SES_small          0.005461
dtype: float64

Let's check out new dataframe out and compare to the old dataframe:

# New way to subset

In [19]:
ind1 = patient.columns.str.endswith('small')
df_new = \
    patient[patient.columns[ind1]]
df_old = \
    patient[patient.columns[~ind1]]
## take the time to do an exact memory profile:
df_new.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
patient_id_small     100000 non-null uint32
hosp_id_small        100000 non-null uint8
age_small            100000 non-null uint8
lab_value_1_small    100000 non-null float32
gender_small         100000 non-null category
mortality_small      100000 non-null category
SES_small            100000 non-null category
dtypes: category(3), float32(1), uint32(1), uint8(2)
memory usage: 1.2 MB


In [20]:
df_old.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
patient_id     100000 non-null int64
hosp_id        100000 non-null int64
age            100000 non-null int64
gender         100000 non-null object
mortality      100000 non-null object
SES            100000 non-null object
lab_value_1    100000 non-null float64
lab_value_2    100000 non-null float64
dtypes: float64(2), int64(3), object(3)
memory usage: 21.0 MB


When to use 32 versus 64 

1. how much precision do I need?   
6-8 decimal points 64 more accurate. 
2. How fast does my code need to run?
3. How much memory am I using? 

### Inplace operations

In general, Pandas is quite lazy with generating copies of dataframes and will carelessly create copies when we do most operations. If we want to limit this copying behaviour, there are a few things we can do. 

*  Ensure we use an index method when slicing / updating date  
.loc or .iloc not make copy but touch the data directly
*  use the inplace = True argument for some dataframe manipulations

In [21]:
## example of updating age to mask older people for privacy
## no copy created
patient.loc[patient['age'] > 89,'age'] = \
    np.NaN

In [22]:
## use inplace = True if possible examples
df_new
df_new.dropna(inplace=True)
df_new.sort_values(by='age_small',inplace=True)
#df_new.rename(str.upper, axis='columns', inplace=True)
df_new

,patient_id_small,hosp_id_small,age_small,lab_value_1_small,gender_small,mortality_small,SES_small
0,100000,77,70,139.949997,Male,Yes,Med
1,100001,45,64,121.459999,Male,No,Low
2,100002,5,76,53.639999,Male,No,Med
3,100003,13,51,99.750000,Male,No,High
4,100004,70,76,118.639999,Male,No,Low
...,...,...,...,...,...,...,...
99995,199995,60,66,90.879997,Male,No,Med
99996,199996,14,71,105.570000,Male,No,Low
99997,199997,39,73,106.519997,Male,Yes,Med
99998,199998,21,60,78.019997,Female,No,High


/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,patient_id_small,hosp_id_small,age_small,lab_value_1_small,gender_small,mortality_small,SES_small
2078,102078,38,22,62.830002,Male,No,Low
8941,108941,44,23,128.440002,Male,Yes,High
66410,166410,4,25,115.690002,Male,Yes,Med
30714,130714,31,26,44.759998,Male,No,Med
19131,119131,64,27,127.370003,Male,No,Low
...,...,...,...,...,...,...,...
56709,156709,54,105,116.209999,Male,No,High
10129,110129,72,107,80.470001,Male,No,Med
14797,114797,61,107,102.309998,Male,Yes,Med
19396,119396,35,108,159.820007,Male,No,Med


### Sparse Matrices

It is also worth mentioning the support for sparse matrices that Pandas includes. As an example, we may have a large amount of zero's in a matrix of disease conditions. We can encode this as a sparse matrix, where we only denote the non-zero values (or potentially non-missing). In this case, we have three variables that occur ~3% of the time. Why do we need to store all those zeros?!?


Let's generate the data first:

In [23]:
## generate some data
sample_data = \
    pd.DataFrame(np.random.binomial(n=1,p=0.05,size=(10000,3)))
## add an id
sample_data['PatientID'] = np.arange(1,10001,1)
## view
sample_data

,0,1,2,PatientID
0,0,0,0,1
1,0,0,0,2
2,0,0,0,3
3,0,1,0,4
4,0,0,1,5
...,...,...,...,...
9995,0,0,0,9996
9996,0,0,0,9997
9997,0,0,0,9998
9998,0,0,0,9999


In [24]:
## check the size
sample_data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
0            10000 non-null int64
1            10000 non-null int64
2            10000 non-null int64
PatientID    10000 non-null int64
dtypes: int64(4)
memory usage: 312.6 KB


In [25]:
sample_data.to_sparse(fill_value=0)

,0,1,2,PatientID
0,0,0,0,1
1,0,0,0,2
2,0,0,0,3
3,0,1,0,4
4,0,0,1,5
...,...,...,...,...
9995,0,0,0,9996
9996,0,0,0,9997
9997,0,0,0,9998
9998,0,0,0,9999


In [26]:
## check the size
sample_data.info(memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
0            10000 non-null int64
1            10000 non-null int64
2            10000 non-null int64
PatientID    10000 non-null int64
dtypes: int64(4)
memory usage: 312.6 KB


Note this is now 1/3 the original size!

Available sparse dtypes include float64, int64, and bool.

## Chunkwise data processing

Another technique for efficiently working with data is to only work on part of the dataset at a time. Pandas allows us to use the chunk_size option on file import to get a 'generator' from the object. 

In [28]:
## ingest the patient data
patient = pd.read_csv('patient_demo.csv',chunksize=1000)
## This is now a generator object!
patient

We can then work on the dataset in increments, and save our result back out somewhere else!

In [29]:
## implement for loop
for i,data in enumerate(patient):
    if i > 10:
        break
    data.to_csv('patient_{}.csv'.format(i))